## Definindo bibliotecas

Definição das bibliotecas necessárias para o projeto:

1. Pandas: responsável pela manipulação das planilhas em excel (ou dataframes);
2. Numpy: responsável pela manipulação matemática dentro dos dataframes;
3. Datetime: responsável pela manipulação de datas;
4. OS: responsável pela manipulação de recursos do windows;

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import os

## Parametrizando usuário

Através da biblioteca OS a função .getlogin() retorna o usuário do computador que está rodando o código.

In [ ]:
user = os.getlogin()
user

## Definindo diretório dos arquivos em excel

A função .path.join da biblioteca OS irá retornar o caminho do diretório onde estarão os arquivos excel que serão lidos.

In [ ]:
dir1 = os.path.join(user, 'Documentos', 'compilador-de-arquivos-excel')
dir1

## Tratamento de dados dentro dos arquivos primários

### Abertura do excel em um dataframe

Através do pandas, formar um dataframe com o arquivo que será modificado

# Entradas e Saídas

In [ ]:
EntradasSaidas_df = pd.read_excel('Fiscal_ExMG.xlsx', engine='openpyxl', sheet_name='Entradas_Saídas')
display(EntradasSaidas_df)

# PIS e COFINS

In [ ]:
PIS_COFINS_df = pd.read_excel('Fiscal_ExMG.xlsx', engine='openpyxl', sheet_name='PIS_COFINS')
display(PIS_COFINS_df)

### Manipulando a data da coluna movimento

É necessário que o formato final da data nessa coluna seja: d/m/y

In [ ]:
PIS_COFINS_df['Movimento'] = pd.to_datetime(PIS_COFINS_df['Movimento'], format='%Y-%m-%d', errors='coerce')
PIS_COFINS_df['Movimento'] = PIS_COFINS_df['Movimento'].astype('datetime64[ns]')
PIS_COFINS_df['Movimento'] = PIS_COFINS_df['Movimento'].dt.strftime("%d/%m/%Y")
PIS_COFINS_df['Movimento']

In [ ]:
EntradasSaidas_df['Movimento'] = pd.to_datetime(EntradasSaidas_df['Movimento'], format='%Y-%m-%d', errors='coerce')
EntradasSaidas_df['Movimento'] = EntradasSaidas_df['Movimento'].astype('datetime64[ns]')
EntradasSaidas_df['Movimento'] = EntradasSaidas_df['Movimento'].dt.strftime("%d/%m/%Y")
EntradasSaidas_df['Movimento']

## Soma de formulário com nomes vazios

### PIS e COFINS

Somar valores de "PIS Débito" e "COFINS Débito" que são de "Formulário: Venda PDV" na mesma data de "Movimento" e da mesma "Empresa".

Alterando a coluna de movimento para formatação de data que será utilizado posteriormente

#### Atribuindo Nome Em formulários sem valor atribuido.

In [ ]:
PIS_COFINS_df['Formulário'] = PIS_COFINS_df['Formulário'].fillna('Vendas PDV')
display(PIS_COFINS_df)


#### Destacando Formulários, Empresas, Movimento,  COFINS Débito e PIS Débito em relação às Vendas PDV do DataFrame.

In [ ]:
VendasPDV_PisCofins_df = PIS_COFINS_df.loc[PIS_COFINS_df['Formulário'] == 'Vendas PDV', ['Formulário',
                                                                                'Tipo E/S', 'Empresa',
                                                                                'Movimento', 'Emissão', 
                                                                                'COFINS Débito', 'PIS Débito']]                                           
display(VendasPDV_PisCofins_df)


#### Somando COFINS Débito e PIS Débito de mesma data de Movimento e Empresa.

In [ ]:
SomaVendasPDV_PisCofins_df = VendasPDV_PisCofins_df.groupby(['Formulário', 'Tipo E/S', 
                                                    'Empresa', 'Movimento' 
                                                    ])[['COFINS Débito', 'PIS Débito']].sum().reset_index()
display(SomaVendasPDV_PisCofins_df)

### Entradas e Saídas

Somar valor contábil e ICMS crédito/débito que são de "Formulário: Venda PDV" na mesma data de "Movimento" e da mesma "Empresa"

Alterando a coluna de movimento para formatação de data que será utilizado posteriormente

In [ ]:
vendas_pdv_df = EntradasSaidas_df.loc[EntradasSaidas_df['Formulário'] == 'Vendas PDV',
                                 ["Formulário", "Tipo E/S", "Movimento", "Empresa", "Valor Contábil", 
                                  "ICMS Crédito / Débito"]]

display(vendas_pdv_df)

vendas_pdv_3_df = vendas_pdv_df.groupby(['Formulário',
                                         'Tipo E/S',
                                         'Empresa',
                                         'Movimento'])[['Valor Contábil', 'ICMS Crédito / Débito']].sum().reset_index()

display(vendas_pdv_3_df)

### Condição para arquivo final

#### dePara_VlrContábil_e_ICMS

Vendas PDV: contábil credito: 319, contabil debito 37
ICMS s/ Vendas PDV: contabil credito: 243, contabil debito 327
PIS s/ Vendas PDV: credito 246, debito 329
COFINS s/ Vendas PDV: credito 241, debito 326
Nota de Devolução de Compras + NF e coluna numero + coluna nome: credito 80, debito 204
    
Digitação de NF de Faturamento + NF e coluna numero + coluna nome (saída):  
    Se CFOPS 5927, debito 592, credito 80
    Se CFOPS diferente de 5927 mas o NOME é DrogaEx, credito 80, debito 92
    Se CFOPS diferente de tudo isso, credito 341, debito 144
    
ICMS s/ Digitação de NF de Faturamento + NF e coluna numero + coluna nome (saída): credito 243, debito 347

Digitação de NF de Faturamento + NF e coluna numero + coluna nome (entrada):  
    Se o NOME é DrogaEx, credito 93, debito 80
    Se o NOME diferente de DrogaEX, credito 10000, debito 20000
    
ICMS s/ Digitação de NF de Faturamento + NF e coluna numero + coluna nome (entradas): credito 80, debito 59

Devolução de Produtos Caixas + NF e coluna numero + coluna nome: credito 41, debito 332
ICMS s/ Devolução de Produtos Caixas + NF e coluna numero + coluna nome: credito 80, debito 59

Nota Fiscal de Compra e Outras Entradas + NF e coluna numero + coluna nome: 
    Se o NOME é DBR ou conter, credito 10055 e debito 374
    Se o NOME é ou contém BETOFARMA, DELMAR, DOTTO, DEMAC, DROGAEX, EX NG, POA, METROFARMA,    MIYAFARMA, NOVA CAIEIRAS e DROGAROMERO, credito 217 e debito 80
    Se o NOME é diferente disso tudo, credito 204, debito 80
    
ICMS s/ Nota Fiscal de Compra e Outras Entradas + NF e coluna numero + coluna nome: credito 80, debito 59

#### dePara_PIS_e_COFINS

Tudo que for TIPO E/S: E, PIS (debito 63 e credito 80) e COFINS (debito 57 e credito 80)

Tudo que for TIPO E/S: S, PIS (debito 63 e credito 80) e COFINS (debito 57 e credito 80)
    PIS: Digitação de NF de Faturamento + NF e coluna numero + coluna nome (saída): credito 246, debito 349
    COFINS: Digitação de NF de Faturamento + NF e coluna numero + coluna nome (saída): credito 241, debito 346